# transformer-helpers.ipynb

> Code that helps run and inspect parts of the transformer model.

In [ ]:
#| default_exp models.transformer_helpers

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
from typing import Dict, Iterable, Tuple

In [ ]:
#| export
from fastcore.test import *
import torch

In [ ]:
#| export 
from transformer_experiments.models.transformer import (
    block_size,
    Block,
    n_head,
    n_embed,
    TransformerLanguageModel
)
from transformer_experiments.tokenizers.char_tokenizer import CharacterTokenizer


In [ ]:
# Not exported because only used for testing within this notebook
from transformer_experiments.datasets.tinyshakespeare import (
    TinyShakespeareDataSet,
)
from transformer_experiments.trained_models.tinyshakespeare_transformer import (
    create_model_and_tokenizer
)

In [ ]:
# Create a model for testing
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device is {device}")

ts = TinyShakespeareDataSet(cache_file='../artifacts/input.txt')
m, tokenizer = create_model_and_tokenizer(
    saved_model_filename='../artifacts/shakespeare.pt',
    dataset=ts,
    device=device,
)

device is cpu


## Helper functions for encoding/decoding embeddings

In [ ]:
# | export
class EncodingHelpers:
    def __init__(
        self, m: TransformerLanguageModel, tokenizer: CharacterTokenizer, device: str
    ):
        self.m = m
        self.tokenizer = tokenizer
        self.device = device

    def tokenize_string(self, s: str) -> torch.Tensor:
        """Given a string, returns a tensor representing the tokenized string.
        The returned tensor has shape (1, T) where T is the number of tokens,
        so it works in situations that expect a batch dimension."""
        return torch.tensor(
            [self.tokenizer.encode(s)], dtype=torch.long, device=self.device
        )
    
    def stringify_tokens(self, tokens: torch.Tensor) -> str:    
        """Given a tensor of tokens, returns a string representing the tokens."""
        return self.tokenizer.decode(tokens.tolist())

    def embed_string(self, s: str) -> torch.Tensor:
        """Given a string, performs the token and positional embeddings
        done at the beginning of the model and returns the tensor that
        would be sent into the stack of blocks."""
        return self.embed_tokens(self.tokenize_string(s))

    def embed_tokens(self, tokens: torch.Tensor) -> torch.Tensor:
        """Given a tensor of tokens containing a batch of tokens (shape B, T),
        performs the token and positional embeddings done at the beginning of
        the model and returns the tensor that would be sent into the stack of
        blocks."""
        idx = tokens[:, -block_size:]

        # Logic from the model's forward() function
        B, T = idx.shape
        token_emb = self.m.token_embedding_table(idx)
        pos_emb = self.m.position_embedding_table(
            torch.arange(T, device=self.device)
        )  # (T, n_embed)
        x = token_emb + pos_emb
        return x.detach()

In [ ]:
# Tests for encoding helpers
encoding_helpers = EncodingHelpers(m, tokenizer, device)
tokenized = encoding_helpers.tokenize_string('hello')
test_eq(tokenized.shape, (1, 5))
test_eq(tokenized, torch.tensor([[46, 43, 50, 50, 53]]))

stringified = encoding_helpers.stringify_tokens(tokenized[0])
test_eq(stringified, 'hello')

embedded = encoding_helpers.embed_string('hello')
test_eq(embedded.shape, (1, 5, n_embed))

embedded_from_tokens = encoding_helpers.embed_tokens(tokenized)
test_eq(embedded_from_tokens.shape, (1, 5, n_embed))
test_eq(embedded, embedded_from_tokens)


## Helpers for Running Selective Parts of the Model
The functions in this section enable running pieces of the model in isolation and introspecting their intermediate results. This is useful for debugging and understanding the model.

In [ ]:
# | export
class InputOutputAccessor:
    def __init__(self, activations: Dict[str, Tuple]):
        self.activations = activations

    def inputs(self, name: str) -> Tuple[torch.Tensor]:
        return self.activations[name][0]

    def input(self, name: str) -> torch.Tensor:
        inps = self.inputs(name)
        assert len(inps) == 1
        return inps[0]

    def output(self, name: str) -> torch.Tensor:
        return self.activations[name][1]

In [ ]:
# | export
class TransformerAccessors:
    """Class that provides methods for running pieces of a `TransformerLanguageModel`
    in isolation and introspecting their intermediate results."""
    def __init__(self, m: TransformerLanguageModel):
        self.m = m

    def copy_block_from_model(self, block_idx: int):
        """Given the index of a block in the model [0, n_layer), creates
        a new block with identical parameters.

        Returns
        -------
        Tuple
            First element is the new block, second is an `InputOutputAccessor` that
            provides access to the inputs and outputs of the block itself, it top-level
            sub-modules, and children of the self-attention sub-module.
        """
        block = self.m.blocks[block_idx]
        new_block = Block(n_embed, n_head)
        new_block.load_state_dict(block.state_dict())
        new_block.eval()

        activations = {}

        def log_activation_hook(name):
            def hook(_, input, output):
                inputs = tuple([inp.detach() for inp in input])
                activations[name] = (inputs, output.detach())

            return hook

        new_block.register_forward_hook(log_activation_hook('.'))
        for name, module in new_block.named_children():
            module.register_forward_hook(log_activation_hook(name))

        # Register the hook for the self-attention layer's children as
        # I will need this. Wanted this function to not have to know
        # about the internal structure of a block (i.e. not access members
        # by a specific name), so this is unfortunate, but the most expedient.
        for name, module in new_block.sa.named_children():
            module.register_forward_hook(log_activation_hook(f'sa.{name}'))

        return new_block, InputOutputAccessor(activations)

In [ ]:
# Test block copying
accessors = TransformerAccessors(m)
block_idx = 0
old_b = m.blocks[block_idx]
new_b, io_accessor = accessors.copy_block_from_model(block_idx)
test_eq(new_b is old_b, False)

x = encoding_helpers.embed_string('Citizen')
old_result = old_b(x).detach()
new_result = new_b(x).detach()
test_eq(old_result, new_result)
test_eq(io_accessor.input('.'), x)
test_eq(io_accessor.output('.'), new_result)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()